# Breakpoints

In [ ]:
include("stuff.jl")

In [ ]:
cube_break_points = loadCube(joinpath(cubes_base_dir, "cube_break_points"))
cube_pca_online = loadCube(joinpath(cubes_base_dir, "cube_pca_online"))
cube_base_clean = loadCube(joinpath(cubes_base_dir, "cube_base_clean"))
cube_base = getCubeData(Cube(data_cube_dir), variable = vars, time = (Date("2001-01-01"), Date("2011-12-31")))

In [ ]:
plotMAP(cube_base_clean)

In [ ]:
showVarInfo(cube_base)

In [ ]:
plotMAP(cube_pca_online)

In [ ]:
lat_ax = getAxis(LatAxis,  cube_break_points)
lon_ax = getAxis(LonAxis,  cube_break_points)
t_ax   = getAxis(TimeAxis, cube_pca_online)

In [ ]:
R"""
library(viridis)
"""

In [ ]:
R"""
data(coastsCoarse, package = "rworldmap")

from_to <- 120:670
#from_to <- 1:720

elem_col = "black" # "gray50"

lat_ax <- $(lat_ax.values |> reverse |> collect)[from_to]
lon_ax <- $(lon_ax.values |> collect)
t_ax   <- $(t_ax.values |> collect)

bp_data_pc1 <- $(
    cube_break_points[1,:,:] |> 
    x -> convert(Matrix{Union{Float64, Missing}}, x) |> 
    x -> x[:, end:-1:1]
)[,from_to]

bp_data_pc2 <- $(
    cube_break_points[2,:,:] |> 
    x -> convert(Matrix{Union{Float64, Missing}}, x) |> 
    x -> x[:, end:-1:1]
)[,from_to]

bp_data_pc3 <- $(
    cube_break_points[3,:,:] |> 
    x -> convert(Matrix{Union{Float64, Missing}}, x) |> 
    x -> x[:, end:-1:1]
)[,from_to]


n_lv1 <- length(na.exclude(unique(as.vector(c(bp_data_pc1)))))
n_lv2 <- length(na.exclude(unique(as.vector(c(bp_data_pc2)))))
n_lv3 <- length(na.exclude(unique(as.vector(c(bp_data_pc3)))))
max_n_lv <- max(n_lv1, n_lv2, n_lv3)

#col_pal <- brewer.pal(max(n_lv1, n_lv2), "Blues")
col_pal <- c("grey95", rev(viridis(length(t_ax))))
#col_pal <- c(rev(viridis(length(t_ax) + 1 )))

########################
pdf($(joinpath(fig_path, "n_bp_pc1_pc2.pdf")), width = 12, height = 12 * length(from_to) / 720 * 3/2)

layout(matrix(1:3, nrow = 3))

par(mar = c(0, 0, 0, 0))

image(
    lon_ax, lat_ax,
    bp_data_pc1,
    col = col_pal[seq_len(n_lv1)],
    axes = FALSE,
    xlab = NA, ylab = NA,
    useRaster = TRUE
)
plot(coastsCoarse, add = TRUE)

image(
    lon_ax, lat_ax,
    bp_data_pc2,
    col = col_pal[seq_len(n_lv2)],
    axes = FALSE,
    xlab = NA, ylab = NA,
    useRaster = TRUE
)
plot(coastsCoarse, add = TRUE)

image(
    lon_ax, lat_ax,
    bp_data_pc3,
    col = col_pal[seq_len(n_lv3)],
    axes = FALSE,
    xlab = NA, ylab = NA,
    useRaster = TRUE
)
plot(coastsCoarse, add = TRUE)

par(fig = c(0.09, 0.11, 0.01, 0.25), new = TRUE, xpd = NA)
image(
    1:2, 
    seq_len(max_n_lv) - 1, 
    t(1:max_n_lv), 
    col = col_pal,
    cex.lab = 1.5,
    col.lab = elem_col,
    ylab = "year",
    axes = FALSE, 
    bty = "n",
    useRaster = TRUE,
    mgp = c(4.1, 1, 0)    
)
#box(col = elem_col)

# this is just to clean the axis scales so that we can plot a nice x axis
par(new = TRUE)
plot(seq_along(t_ax), t_ax, type = "n", bty = "n", axes = FALSE, xlab = NA, ylab = NA, yaxs = "i")
axis.Date(2, t_ax, col = elem_col, col.axis = elem_col, las = 1, cex.axis = 1.5)

#axis(2, las = 1, col = elem_col, col.axis = elem_col)

grid.text("(a)", 0,   1, just = c(0, 1), gp = gpar(cex = 2))
grid.text("(b)", 0, 2/3, just = c(0, 1), gp = gpar(cex = 2))
grid.text("(c)", 0, 1/3, just = c(0, 1), gp = gpar(cex = 2))

dev.off()
"""

show_pdf(
    "http://127.0.0.1:8333/files/results/fig/n_bp_pc1_pc2.pdf",
    width = 700,
    height = 700
)

In [ ]:
plot(
    ntuple(
        i -> cube_n_break_points[i, :, :] |> 
            x -> convert(Array, x) |> 
            missing_to_nan |> 
            rotl90 |> 
            x -> heatmap(x, title = "# of bp in PC $i"), 
        size(cube_n_break_points)[1]
    )..., 
    fmt = :png,
    layout = (6, 2),
    size = (1500, 2500)
)